## 準備
依存関係をインストールします

In [ ]:
!pip install -U pip
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers["ja"] numpy pandas sentencepiece

## 学習データのダウンロード
今回は株式会社リクルートが提供する"Japanese Realistic Textual Entailment Corpus" (``https://github.com/megagonlabs/jrte-corpus``)を学習データとして利用します  
**このデータセットライセンスは `CC BY-NC-SA 4.0` なので営利目的には利用できません。**
### 参考
林部祐太．
知識の整理のための根拠付き自然文間含意関係コーパスの構築．
言語処理学会第26回年次大会論文集，pp.820-823. 2020. (NLP 2020)
[[PDF]](https://www.anlp.jp/proceedings/annual_meeting/2020/pdf_dir/P4-9.pdf)
[[Poster]](https://storage.googleapis.com/megagon-publications/nlp2020/p4-9_hayashibe_poster.pdf)

In [ ]:
!wget https://raw.githubusercontent.com/megagonlabs/jrte-corpus/master/data/pn.tsv

ダウンロードしたデータの中身を確認してみます

In [ ]:
!head pn.tsv

ヘッダがないので付与しつつpandasのdata frameとして読み込みます

In [ ]:
import pandas as pd
df = pd.read_csv("pn.tsv", names=("id", "label", "text", "judges", "usage"), sep="\t")
print(df)

## 学習データの分割
学習データを前処理しつつ学習用/検証用/テスト用に分割します

In [ ]:
df = df.replace({"label": {0: 2, 1: 1, -1: 0}})

train_data = []
eval_data = []
test_data = []
for i, row in df.iterrows():
    if row["usage"] == "train":
        train_data.append({"x": row["text"], "y": row["label"]})
    elif row["usage"] == "dev":
        eval_data.append({"x": row["text"], "y": row["label"]})
    elif row["usage"] == "test":
        test_data.append({"x": row["text"], "y": row["label"]})

## モデルの準備
次に、学習に利用するモデルを用意していきます

In [ ]:
from transformers import AlbertTokenizerFast, AlbertForSequenceClassification

label2id = {"Negative": 0, "Positive": 1, "Neutral": 2}
base_model = "ken11/albert-base-japanese-v1"
tokenizer = AlbertTokenizerFast.from_pretrained(base_model)
model = AlbertForSequenceClassification.from_pretrained(base_model, label2id=label2id, id2label={0: "Negative", 1: "Positive", 2: "Neutral"})

## Trainerの準備
TrainingArgumentsを設定し、Trainerを作成していきます  
Trainerにはdata_collatorを渡してあげる必要があるので、data_collatorも作成します  
  
data_collatorはtransformersにすでにあるものを利用することもできますが、ここでは自前で定義していきます

In [ ]:
import torch

def data_collator(features: list) -> dict:
    x = [f["x"] for f in features]
    y = [f["y"] for f in features]
    inputs = tokenizer(x, return_tensors=None, padding='max_length', truncation=True, max_length=128)
    input_labels = []
    for label in y:
        input_labels.append([label])
    inputs['labels'] = input_labels
    batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in inputs.items()}
    return batch

パラメータ類を設定しておきます

In [ ]:
ckpt_dir = "./ckpt"
batch_size = 16
epochs = 3
learning_rate = 3e-5
save_freq = 100
model_output_dir = "./dest"

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(output_dir=ckpt_dir,
                         do_train=True,
                         do_eval=True,
                         do_predict=True,
                         per_device_train_batch_size=batch_size,
                         per_device_eval_batch_size=batch_size,
                         learning_rate=learning_rate,
                         num_train_epochs=epochs,
                         evaluation_strategy="steps",
                         eval_steps=save_freq,
                         save_strategy="steps",
                         save_steps=save_freq,
                         load_best_model_at_end=True,
                        )

In [ ]:
from transformers import Trainer, EarlyStoppingCallback

trainer = Trainer(model=model,
                  args=args,
                  data_collator=data_collator,
                  train_dataset=train_data,
                  eval_dataset=eval_data,
                  callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
                 )

学習を実行します

In [ ]:
trainer.train()

できあがったモデルをテストします

In [ ]:
_, _, metrics = trainer.predict(test_data, metric_key_prefix="test")
print(metrics)

モデルを保存します

In [ ]:
trainer.save_model(model_output_dir)

トークナイザーは特に変更をしていませんが、使うときのために一緒に保存しておきましょう

In [ ]:
tokenizer.save_pretrained(model_output_dir)

## 推論
できあがったモデルを使って推論を行ってみます

In [ ]:
import numpy as np

def inference(text: str):
    model = AlbertForSequenceClassification.from_pretrained(model_output_dir)
    tokenizer = AlbertTokenizerFast.from_pretrained(model_output_dir)
    
    inputs = tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, max_length=128)
    pred = model(**inputs).logits[0]
    pred = np.argmax(pred.detach().numpy(), axis=-1)
    print(f"input text: {text}\nsentiment: {model.config.id2label[pred]}")

In [ ]:
inference("駅から近くて便利でした")